In [1]:
import pandas as pd
import os
import numpy as np
import pandas_profiling

# This line is needed to display plots inline in Jupyter Notebook
%matplotlib inline

# Required for basic python plotting functionality
import matplotlib.pyplot as plt

# Required for formatting dates later in the case
import datetime
import matplotlib.dates as mdates

# Required to display image inline
from IPython.display import Image

# Advanced plotting functionality with seaborn
import seaborn as sns
sns.set(style="whitegrid") # can set style depending on how you'd like it to look

import folium  #needed for interactive map
from folium.plugins import HeatMap

import geopandas
from shapely.geometry import Polygon
from shapely.geometry import Point
from geopandas import GeoDataFrame

In [23]:
path = 'C:/Users/johns/Documents/DS4A_workspace/Datathon/DataSet'
file = 'uber_nta_2014_2015.csv'

uber_trips = pd.read_csv(path + '/' + file, sep = ',', doublequote = True)

In [10]:
uber_trips.head()

,Unnamed: 0,pickup_datetime,dispatch_base,nta_code
0,0,2014-04-01 00:11:00,B02512,MN31
1,1,2014-04-01 02:31:00,B02512,MN31
2,2,2014-04-01 05:36:00,B02512,MN31
3,3,2014-04-01 05:52:00,B02512,MN31
4,4,2014-04-01 06:06:00,B02512,MN31


In [24]:
uber_trips.rename({'Unnamed: 0':'index'}, inplace = True, axis = 1)
uber_trips.head()

,index,pickup_datetime,dispatch_base,nta_code
0,0,2014-04-01 00:11:00,B02512,MN31
1,1,2014-04-01 02:31:00,B02512,MN31
2,2,2014-04-01 05:36:00,B02512,MN31
3,3,2014-04-01 05:52:00,B02512,MN31
4,4,2014-04-01 06:06:00,B02512,MN31


In [31]:
uber_trips['pickup_datetime'] = pd.to_datetime(uber_trips['pickup_datetime'], format = '%Y-%m-%d %H:%M:%S')

In [32]:
uber_trips.to_parquet(path + '/Parquet/uber_trips_nta_2014_2015.parquet', engine = 'pyarrow')

In [14]:
path = 'C:/Users/johns/Documents/DS4A_workspace/Datathon/DataSet'
file = 'zones.csv'

zones = pd.read_csv(path + '/' + file, sep = ',', doublequote = True)
boroughs_nta = zones.groupby(['borough', 'nta_code'])['zone'].count().reset_index(name = 'count')
boroughs_nta.head()

,borough,nta_code,count
0,Bronx,BX01,1
1,Bronx,BX03,1
2,Bronx,BX05,1
3,Bronx,BX06,1
4,Bronx,BX07,1


In [25]:
print(uber_trips.shape)
uber_trips = uber_trips.merge(boroughs_nta, how = 'inner', on = 'nta_code')
print(uber_trips.shape)

(18676106, 4)
(18652514, 6)


In [26]:
uber_trips.drop(['count', 'index'], inplace = True, axis = 1)

In [27]:
uber_trips.head()

,pickup_datetime,dispatch_base,nta_code,borough
0,2014-04-01 00:11:00,B02512,MN31,Manhattan
1,2014-04-01 02:31:00,B02512,MN31,Manhattan
2,2014-04-01 05:36:00,B02512,MN31,Manhattan
3,2014-04-01 05:52:00,B02512,MN31,Manhattan
4,2014-04-01 06:06:00,B02512,MN31,Manhattan


In [29]:
uber_trips.borough.value_counts(normalize = 1)

Manhattan        0.739854
Brooklyn         0.156287
Queens           0.090008
Bronx            0.013422
Staten Island    0.000423
EWR              0.000006
Name: borough, dtype: float64